In [1]:
import numpy as np
import pandas as pd

In [2]:
resaleflat1 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
#resaleflat2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
resaleflat3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
resaleflat4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
resaleflat = pd.concat((resaleflat1,resaleflat3,resaleflat4), axis=0, sort=True)
print(resaleflat.shape)
resaleflat.index = range(139788)

resaleflat_l = resaleflat.copy()
resaleflatview = resaleflat1.copy()

(139788, 11)


In [3]:
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [4]:
resaleflat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139788 entries, 0 to 139787
Data columns (total 11 columns):
block                  139788 non-null object
flat_model             139788 non-null object
flat_type              139788 non-null object
floor_area_sqm         139788 non-null float64
lease_commence_date    139788 non-null int64
month                  139788 non-null object
remaining_lease        87585 non-null object
resale_price           139788 non-null float64
storey_range           139788 non-null object
street_name            139788 non-null object
town                   139788 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 11.7+ MB


In [5]:
resaleflat.isnull().sum()

#Remaining lease has a lot of null, need to fix

block                      0
flat_model                 0
flat_type                  0
floor_area_sqm             0
lease_commence_date        0
month                      0
remaining_lease        52203
resale_price               0
storey_range               0
street_name                0
town                       0
dtype: int64

# We had to convert categorical to numerical first to see correlations, now we redo the table using onehotencoding

In [6]:
resaleflat_l.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [7]:
#Convert first two char into integer for years left
resaleflat_l['remaining_lease_year'] = resaleflat_l['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease'].str.slice(start=9,stop=12)

replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].replace(replacemonth)

resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].astype('float') / 12
resaleflat_l['remaining_lease'] = resaleflat_l['remaining_lease_year'] + resaleflat_l['remaining_lease_month']
resaleflat_l = resaleflat_l.drop(['remaining_lease_year','remaining_lease_month'], axis=1)

resaleflat_l.month = resaleflat_l.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_l.remaining_lease[resaleflat_l.remaining_lease.isnull()] = resaleflat_l['month'] - resaleflat_l['lease_commence_date']

#Normalize the month, floor area and remaining lease (timeframe)
resaleflat_l.month = (resaleflat_l.month - resaleflat_l.month.mean()) / resaleflat_l.month.std()
resaleflat_l.floor_area_sqm = (resaleflat_l.floor_area_sqm - resaleflat_l.floor_area_sqm.mean()) / resaleflat_l.floor_area_sqm.std()
resaleflat_l.remaining_lease = (resaleflat_l.remaining_lease - resaleflat_l.remaining_lease.mean()) / resaleflat_l.remaining_lease.std()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#Blocks in Singapore is rather irregular and does not impact on the pricing
#Lease commence date is highly correlated with remaining lease as remaining lease = current date - lease commence date

#Use town instead of street_name for random forest, both are highly correlated so just use one

resaleflat_l = resaleflat_l.drop(columns=['block','lease_commence_date','street_name','flat_type'])

In [9]:
resaleflat_l.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,town
0,Improved,-2.163608,0.540959,0.710285,232000.0,10 TO 12,ANG MO KIO
1,New Generation,-1.225543,0.540959,0.681966,250000.0,01 TO 03,ANG MO KIO
2,New Generation,-1.225543,0.540959,0.751191,262000.0,01 TO 03,ANG MO KIO
3,New Generation,-1.184757,0.540959,0.738605,265000.0,04 TO 06,ANG MO KIO
4,New Generation,-1.225543,0.540959,0.751191,265000.0,01 TO 03,ANG MO KIO


In [10]:
all_flat_model = pd.get_dummies(resaleflat_l['flat_model'], prefix='flt')
all_towns = pd.get_dummies(resaleflat_l['town'], prefix='town')

resaleflat_l = resaleflat_l.drop(columns=['flat_model','town'])
resaleflat_l = pd.concat((resaleflat_l,all_flat_model,all_towns), axis=1)

In [11]:
resaleflat_l.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [12]:
#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_l.storey_range = resaleflat_l.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

resaleflat_l.storey_range = (resaleflat_l.storey_range - resaleflat_l.storey_range.mean()) / resaleflat_l.storey_range.std()

In [13]:
storey_resales = resaleflat_l[['storey_range','resale_price']]
storey_resales.corr()

#Storey range has correlation of 0.30 with resale price

,storey_range,resale_price
storey_range,1.000000,0.303071
resale_price,0.303071,1.000000


In [14]:
lease_resales = resaleflat_l[['remaining_lease','resale_price']]
lease_resales.corr()

#Remaining lease only has -0.067 correlation with resale price

,remaining_lease,resale_price
remaining_lease,1.0000,-0.0671
resale_price,-0.0671,1.0000


In [15]:
month_resales = resaleflat_l[['month','resale_price']]
month_resales.corr()

#Year/month bought (month) has -0.07 correlation with resale price

,month,resale_price
month,1.000000,-0.073186
resale_price,-0.073186,1.000000


In [16]:
floor_resales = resaleflat_l[['floor_area_sqm','resale_price']]
floor_resales.corr()

#Floor area SQM has 0.667 correlation with resale price, very high!

,floor_area_sqm,resale_price
floor_area_sqm,1.000000,0.667404
resale_price,0.667404,1.000000


In [17]:
resaleflat_l.head()

,floor_area_sqm,month,remaining_lease,resale_price,storey_range,flt_2-room,flt_Adjoined flat,flt_Apartment,flt_DBSS,flt_Improved,...,town_PASIR RIS,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN
0,-2.163608,0.540959,0.710285,232000.0,-0.030180,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,-1.225543,0.540959,0.681966,250000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.225543,0.540959,0.751191,262000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.184757,0.540959,0.738605,265000.0,-0.365163,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.225543,0.540959,0.751191,265000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
resaleflat_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139788 entries, 0 to 139787
Data columns (total 52 columns):
floor_area_sqm                139788 non-null float64
month                         139788 non-null float64
remaining_lease               139788 non-null float64
resale_price                  139788 non-null float64
storey_range                  139788 non-null float64
flt_2-room                    139788 non-null uint8
flt_Adjoined flat             139788 non-null uint8
flt_Apartment                 139788 non-null uint8
flt_DBSS                      139788 non-null uint8
flt_Improved                  139788 non-null uint8
flt_Improved-Maisonette       139788 non-null uint8
flt_Maisonette                139788 non-null uint8
flt_Model A                   139788 non-null uint8
flt_Model A-Maisonette        139788 non-null uint8
flt_Model A2                  139788 non-null uint8
flt_Multi Generation          139788 non-null uint8
flt_New Generation            139788 non-null u

In [19]:
resaleflat_l.sum().index.values

array(['floor_area_sqm', 'month', 'remaining_lease', 'resale_price',
       'storey_range', 'flt_2-room', 'flt_Adjoined flat', 'flt_Apartment',
       'flt_DBSS', 'flt_Improved', 'flt_Improved-Maisonette',
       'flt_Maisonette', 'flt_Model A', 'flt_Model A-Maisonette',
       'flt_Model A2', 'flt_Multi Generation', 'flt_New Generation',
       'flt_Premium Apartment', 'flt_Premium Apartment Loft',
       'flt_Premium Apartment.', 'flt_Premium Maisonette',
       'flt_Simplified', 'flt_Standard', 'flt_Terrace', 'flt_Type S1',
       'flt_Type S2', 'town_ANG MO KIO', 'town_BEDOK', 'town_BISHAN',
       'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_BUKIT PANJANG',
       'town_BUKIT TIMAH', 'town_CENTRAL AREA', 'town_CHOA CHU KANG',
       'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL',
       'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
       

In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [21]:
#Seperate datasets

X = resaleflat_l.drop('resale_price', axis=1)
Y = resaleflat_l['resale_price']

print(X.shape)
print(Y.shape)

(139788, 51)
(139788,)


In [22]:
#Since we use town names instead of street names, features not that numerous to be selected, just use all
#We also know that each town has its own house pricing so its good to keep all of them

#Choose top 300 features for model out of 561

# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# X = SelectKBest(f_classif, k=300).fit_transform(X, Y)

In [23]:
print(X.shape)

(139788, 51)


In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Random Forest Regressor

In [25]:
dec_tree = DecisionTreeRegressor(min_samples_split=50, random_state=42)

dec_tree.fit(x_train,y_train)
pred_dec_tree = dec_tree.predict(x_test)

In [26]:
pred_dec_tree

array([311510.41666667, 455969.63636364, 393142.85714286, ...,
       313080.72727273, 330761.42222222, 411222.22222222])

In [27]:
y_test.head()

51472     308000.0
103548    460000.0
63445     381000.0
40489     750000.0
60647     480000.0
Name: resale_price, dtype: float64

In [28]:
(y_test - pred_dec_tree).head()

51472     -3510.416667
103548     4030.363636
63445    -12142.857143
40489     35934.896552
60647     29386.325581
Name: resale_price, dtype: float64

In [29]:
#Let's see how our model performed

import math

math.sqrt(mean_squared_error(y_test, pred_dec_tree, multioutput='uniform_average'))

37557.09403875641

In [30]:
#Find RMSE for baseline model - i.e. take average house price

base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())

base

#RMSE of $138415 for prediction of house price using average

138415.3257061527

In [31]:
dec_tree_diff = math.sqrt(((y_test - pred_dec_tree) ** 2).mean())
dec_tree_diff

#RMSE of $51868 using linear regression
#RMSE of $37557 using decision tree

37557.094038756266

In [32]:
def RMSE(actual,predict):
    base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())
    x = math.sqrt(((actual - predict) ** 2).mean())
    return x/base

RMSE(y_test, pred_dec_tree)

0.2713362400236494

In [33]:
#Get R^2 of prediction in linear regression, 0.860
#Get R^2 for random forest, 0.926

dec_tree.score(x_test,y_test)

0.9263766448498267

In [34]:
y_test.values

array([308000., 460000., 381000., ..., 290000., 310000., 420000.])

In [35]:
pred_dec_tree

array([311510.41666667, 455969.63636364, 393142.85714286, ...,
       313080.72727273, 330761.42222222, 411222.22222222])

In [36]:
result1 = np.concatenate(([y_test.values], [pred_dec_tree],[y_test.values - pred_dec_tree])).T

In [37]:
result1_df = pd.DataFrame(result1, columns=['actual','predict','difference'])

In [38]:
result1_df

,actual,predict,difference
0,308000.0,311510.416667,-3510.416667
1,460000.0,455969.636364,4030.363636
2,381000.0,393142.857143,-12142.857143
3,750000.0,714065.103448,35934.896552
4,480000.0,450613.674419,29386.325581
5,325000.0,340185.142857,-15185.142857
6,385000.0,359192.000000,25808.000000
7,360000.0,393909.238095,-33909.238095
8,350000.0,360634.153846,-10634.153846
9,337000.0,347611.076923,-10611.076923


In [39]:
print(result1_df['difference'].mean())
print(result1_df['difference'].max())
print(result1_df['difference'].min())

-61.407589187720575
550000.0
-304995.3488372093


In [40]:
result1_df[result1_df['difference'] > 100000]

,actual,predict,difference
182,920000.0,701928.740741,218071.259259
216,460000.0,339790.000000,120210.000000
406,660000.0,543683.692308,116316.307692
571,625000.0,489172.324324,135827.675676
615,800000.0,681631.900000,118368.100000
782,800000.0,677994.909091,122005.090909
973,475000.0,372766.857143,102233.142857
996,735000.0,587037.000000,147963.000000
1068,540000.0,439074.051282,100925.948718
1236,500000.0,341328.000000,158672.000000


In [41]:
result1_df[result1_df['difference'] < -100000]

,actual,predict,difference
152,424000.0,586639.534884,-162639.534884
210,283000.0,587995.348837,-304995.348837
247,430000.0,567876.000000,-137876.000000
412,413000.0,549285.714286,-136285.714286
562,680000.0,812750.000000,-132750.000000
761,429000.0,586666.666667,-157666.666667
943,440000.0,567830.666667,-127830.666667
955,455000.0,645988.800000,-190988.800000
1023,417000.0,521333.333333,-104333.333333
1029,510000.0,660582.761905,-150582.761905


In [42]:
print(dict(zip(dec_tree.feature_importances_, x_train.columns)))
print('\n')
feat_importance = dec_tree.tree_.compute_feature_importances(normalize=True)
print("Feat importance = " + str(feat_importance))

#from io import StringIO
#out = StringIO()
#out = export_graphviz(dec_tree, out_file='tree.dot')

# import os
# os.environ["PATH"] += os.pathsep + r'D://Anaconda3//Lib//site-packages//graphviz'

# import pydot
# (graph,) = pydot.graph_from_dot_file(r'C://Users//jx/Desktop//Python Tutorial//TUTORIALS BASICS OF PYTHON//Machine Learning//tree.dot')
# graph.write_png('tree.png')

#from subprocess import check_call
#check_call(['dot','-Tpng','tree.dot','-o','OutputFile.png'])


{0.5210532726441699: 'floor_area_sqm', 0.02372594930259819: 'month', 0.04962334545337873: 'remaining_lease', 0.09723265491404494: 'storey_range', 0.0: 'flt_Premium Apartment Loft', 0.00020237570119001626: 'flt_Adjoined flat', 0.0013506030652119462: 'flt_Apartment', 0.02177566212583225: 'flt_DBSS', 0.005049740614035725: 'flt_Improved', 0.0002296167087355931: 'flt_Improved-Maisonette', 0.0002086498122457365: 'flt_Maisonette', 0.008723285243959525: 'flt_Model A', 0.00012109492131506724: 'flt_Model A-Maisonette', 0.00011688138408264889: 'flt_Model A2', 5.245556477767967e-05: 'flt_Multi Generation', 0.0032412126471323467: 'flt_New Generation', 0.0008474520317393037: 'flt_Premium Apartment', 4.083850566403407e-05: 'flt_Premium Apartment.', 0.00013914940897870177: 'flt_Premium Maisonette', 0.0001716940974764504: 'flt_Simplified', 0.0009394610298784324: 'flt_Standard', 0.0024195668190476255: 'flt_Terrace', 0.00012459334731895302: 'flt_Type S1', 0.000826207844971679: 'flt_Type S2', 0.0096633054

In [51]:
#If we take resale flat from 2000-2012, you can see the model does very poorly with this set

resaleflat_new = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')

resaleflat_new.month = resaleflat_new.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_new['remaining_lease'] = resaleflat_new['month'] - resaleflat_new['lease_commence_date']

#Normalize the month, floor area and remaining lease (timeframe)
resaleflat_new.month = (resaleflat_new.month - resaleflat_new.month.mean()) / resaleflat_new.month.std()
resaleflat_new.floor_area_sqm = (resaleflat_new.floor_area_sqm - resaleflat_new.floor_area_sqm.mean()) / resaleflat_new.floor_area_sqm.std()
resaleflat_new.remaining_lease = (resaleflat_new.remaining_lease - resaleflat_new.remaining_lease.mean()) / resaleflat_new.remaining_lease.std()

resaleflat_new = resaleflat_new.drop(columns=['block','lease_commence_date','street_name','flat_type'])

all_flat_model = pd.get_dummies(resaleflat_new['flat_model'], prefix='flt')
all_towns = pd.get_dummies(resaleflat_new['town'], prefix='town')

resaleflat_new = resaleflat_new.drop(columns=['flat_model','town'])
resaleflat_new = pd.concat((resaleflat_new,all_flat_model,all_towns), axis=1)

#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_new.storey_range = resaleflat_new.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

resaleflat_new.storey_range = (resaleflat_new.storey_range - resaleflat_new.storey_range.mean()) / resaleflat_new.storey_range.std()

In [52]:
#Substitue columns from training table not present in external testing table (filled with 0s)
for col in resaleflat_l.columns:
    try:
        resaleflat_new[col] = resaleflat_new[col]
    except:
        resaleflat_new[col] = 0

resaleflat_new.shape
        
x_test2 = resaleflat_new.drop(columns='resale_price')
y_test2 = resaleflat_new['resale_price']


In [53]:
pred_dec_tree2 = dec_tree.predict(x_test2)

#Find RMSE for baseline model - i.e. take average house price

base2 = math.sqrt(((y_test2 - y_test2.mean()) ** 2).mean())
print(base2)

#RMSE of $138415 for prediction of house price using average

dec_tree_diff2 = math.sqrt(((y_test2 - pred_dec_tree2) ** 2).mean())
print(dec_tree_diff2)

#RMSE of $51868 using linear regression
#RMSE of $37557 using decision tree

def RMSE(actual,predict):
    base = math.sqrt(((actual - actual.mean()) ** 2).mean())
    x = math.sqrt(((actual - predict) ** 2).mean())
    return x/base

print(RMSE(y_test2, pred_dec_tree2))

print(dec_tree.score(x_test2,y_test2))

112118.81555046461
278638.4403774629
2.4852067782686116
-5.176252730753291
